In [ ]:
import os
import zipfile
from xml.dom import minidom
import os
import csv
import pandas as pd
from pandas import DataFrame


# Identify secondary studies
# For primary studies (and single paper studies, not explicitly marked as primary)
# get the characteristics from the Cochrane table
# get criteria for considering studies for this review from Methods

In [ ]:
RCT_filtered_df = pd.read_csv("7226_Articles_withPMIDs.csv")
print (len(RCT_filtered_df))
display(RCT_filtered_df.head(2))

In [ ]:
review_names_list = []
study_id_list = []
primary_yes_no_list = []
reference_title_list = []
reference_author_list = []
reference_primary_list = []
primary_study_single_reference_list_2 = []
primary_study_multiple_references_list_2 = []

review_names_list_1 = []
study_id_list_1 = []
study_name_list_1 = []
study_year_list_1 =[]
count_references_list_1 = []
primary_study_single_reference_list_1 = []
primary_study_multiple_references_list_1 = []

RCT_filtered_df["Count_Reference"] = ""
RCT_filtered_df["Primary_Study_Single_Reference"] = ""
RCT_filtered_df["Has_Primary_Reference_Marked"] = ""


for index, row in RCT_filtered_df.iterrows(): 
    if index % 200 == 0:
        print (index)
    review_name_273 = row["Cochrane Review"]
    study_id_273 = row["Study ID"]
    title_273 = row["Title"]
    
    with zipfile.ZipFile('/efs/lhoang2/Cochrane_Reviews/Cochrane_Reviews_2018.zip') as z:
        for filename in z.namelist():
            if not os.path.isdir(filename):
                if (review_name_273 in filename):
                    with z.open(filename) as f:
                        doc = minidom.parse(f) 

                        # doc.getElementsByTagName returns NodeList
                        included_studies = doc.getElementsByTagName("INCLUDED_STUDIES")

                        if (included_studies.length >0):
                            studies = included_studies[0].getElementsByTagName("STUDY")

                        for study in studies:

                            study_id = study.getAttribute("ID")
                            
                            if study_id == study_id_273:
                                study_name = study.getAttribute("NAME")
                                study_year = study.getAttribute("YEAR")

                                review_names_list_1.append(filename)
                                study_id_list_1.append(study_id)
                                study_name_list_1.append(study_name)
                                study_year_list_1.append(study_year)

                                references = study.getElementsByTagName("REFERENCE")
                                count_reference = len(references)
                                count_references_list_1.append(count_reference)

                                if count_reference == 1:
                                    primary_study_single_reference = "YES"
                                else:
                                    primary_study_single_reference = "NO"
                                primary_study_single_reference_list_1.append(primary_study_single_reference)
                                
                                if (references.length >0): 
                                    primary_study_multiple_references = "NO"
                                    
                                    for reference in references:

                                        reference_title = reference.getElementsByTagName("TI")
                                        if (reference_title.length > 0):
                                            reference_title = reference_title[0].firstChild.data
                                        else:
                                            reference_title = "None"
                                            
                                        if (title_273 == reference_title):

                                            #getting authors information
                                            authors = reference.getElementsByTagName("AU")
                                            if (authors.length >0):
                                                authors = authors[0].firstChild.data
                                            else:
                                                authors = "None"
                                            
                                            #getting primary information
                                            primary_yes_no = reference.getAttribute("PRIMARY")
                                            
#                                             print ("test: ", study_id, reference_title, primary_yes_no)

                                            if primary_yes_no == "YES":
                                                primary_study_multiple_references = "YES"
                                            

                                            review_names_list.append(filename)
                                            study_id_list.append(study_id)
                                            reference_title_list.append(reference_title)
                                            reference_author_list.append(authors)
                                            reference_primary_list.append(primary_yes_no)
                                            
                                            primary_study_single_reference_list_2.append(primary_study_single_reference)
                                            primary_study_multiple_references_list_2.append(primary_study_multiple_references)
                                            
#                                 primary_study_multiple_references_list_1.append(primary_study_multiple_references)
                                
#                                 RCT_filtered_df.at[index,"Count_Reference"] = count_reference
#                                 RCT_filtered_df.at[index,"Primary_Study_Single_Reference"] = primary_study_single_reference
#                                 RCT_filtered_df.at[index,"Has_Primary_Reference_Marked"] = primary_study_multiple_references
                            
                        
                         
                        
                                
reference_df = DataFrame({'Review_Name': review_names_list , 
                'Study_ID': study_id_list,
                'Reference_Title': reference_title_list,
                'Reference_Authors':reference_author_list,
                'Reference_primary_Yes_No':reference_primary_list,
                'Single_Reference_Study_Primary': primary_study_single_reference_list_2,
                'Multiple_Reference_Study_Has_Primary_Marked': primary_study_multiple_references_list_2})
print ("REFERENCE DATAFRAME: ")
print (len(reference_df))
display(reference_df.head(2))                                            

In [ ]:
reference_df.to_csv("7226_articles_with_primaryarticle_info_12022021.csv")

In [ ]:
data_file = "analyzing_the_273.csv"
data_df = pd.read_csv(data_file)
print (len(data_df))
print (len(RCT_filtered_df))

data_df = data_df.sort_values(by=['Study ID'])
RCT_filtered_df = RCT_filtered_df.sort_values(by=['Study ID'])

print (data_df.columns)
print (RCT_filtered_df.columns)

display(data_df.head(2))
display(RCT_filtered_df.head(2))

merged_df = pd.merge(
    data_df_Jodi,
    RCT_filtered_df,
    how="inner",
    on=["Study ID","Title"],
    left_index=False,
    right_index=False)

print (len(merged_df))
display(merged_df.head(5))

merged_df.to_csv("273_LowScoresStudies_withPrimaryStudiesInfo.csv")

In [ ]:
RCT_filtered_df.to_csv("273_LowScoresStudies_withPrimaryStudiesInfo_11182021.csv")
display(RCT_filtered_df.head(3))

In [ ]:
review_names_list = []
study_id_list = []
study_method_list = []
study_participants_list = []
study_intervention_list = []
study_outcomes_list = []
study_notes_list = []

reviews = list(set(RCT_filtered_df["Cochrane Review"]))
reviews_new =[]
for item in reviews:
    item_new = item[:16]
    reviews_new.append(item_new)
print (len(reviews_new))

with zipfile.ZipFile('/efs/lhoang2/Cochrane_Reviews/Cochrane_Reviews_2018.zip') as z:
    for filename in z.namelist():
        if not os.path.isdir(filename):
            if (filename):
                filename_split = filename.split("/")
                cochrane_name = filename_split[2]
                cochrane_name_short = cochrane_name[:16]
#                 print (cochrane_name_short)
                if cochrane_name_short in reviews_new:
                    review_study_df = RCT_filtered_df[RCT_filtered_df["Cochrane Review"].str.contains(cochrane_name_short)]
                    study_ids = list(review_study_df["Study ID"])

                    with z.open(filename) as f:
                        doc = minidom.parse(f) 

                        # doc.getElementsByTagName returns NodeList
                        char_included_study = doc.getElementsByTagName("INCLUDED_CHAR")
                        for i in range (0, len(char_included_study)):
                            study_id = (char_included_study[i].getAttribute("STUDY_ID"))
                            if study_id in study_ids:
                                print (study_id)
                                study_method = ""
                                study_participants = ""
                                study_intervention = ""
                                study_outcome = ""
                                study_characteristics_notes = ""

                                if (study_id != ""):
                                    char_methods = (char_included_study[i].getElementsByTagName("CHAR_METHODS")[0].childNodes)
                                    if len(char_methods) >=1:
                                        for char_method in char_methods:                                        
                                            if char_method.nodeType == minidom.Node.ELEMENT_NODE:
                                                nodes =  (char_method.childNodes)
                                                for child in nodes:
                                                    if (child.nodeType == minidom.Node.ELEMENT_NODE):
                                                        nodes_level_1 = child.childNodes
                                                        for child_level_1 in nodes_level_1:
                                                            if (child_level_1.nodeType == minidom.Node.TEXT_NODE):
                                                                text = (child_level_1.data)
                                                                text  = text.replace("\n","")
                                                                study_method += " " + text
                                                    elif (child.nodeType == minidom.Node.TEXT_NODE):
                                                        text = (child.data)
                                                        text  = text.replace("\n","")
                                                        study_method += " " + text
                                    else:
                                        study_method = "N/A"
                                    
                                    if len(char_included_study[i].getElementsByTagName("CHAR_PARTICIPANTS")) >= 1:
                                        char_participants = (char_included_study[i].getElementsByTagName("CHAR_PARTICIPANTS")[0].childNodes)
                                        if len(char_participants) >=1:
                                            for char_participant in char_participants:                                        
                                                if char_participant.nodeType == minidom.Node.ELEMENT_NODE:
                                                    nodes =  (char_participant.childNodes)
                                                    for child in nodes:
                                                        if (child.nodeType == minidom.Node.ELEMENT_NODE):
                                                            nodes_level_1 = child.childNodes
                                                            for child_level_1 in nodes_level_1:
                                                                if (child_level_1.nodeType == minidom.Node.TEXT_NODE):
                                                                    text = (child_level_1.data)
                                                                    text  = text.replace("\n","")
                                                                    study_participants += " " + text
                                                        elif (child.nodeType == minidom.Node.TEXT_NODE):
                                                            text = (child.data)
                                                            text  = text.replace("\n","")
                                                            study_participants += " " + text
                                        else:
                                            study_participants = "N/A"
                                    else:
                                        study_participants = "N/A"

                                    if len(char_included_study[i].getElementsByTagName("CHAR_INTERVENTIONS")) >=1:
                                        char_interventions = (char_included_study[i].getElementsByTagName("CHAR_INTERVENTIONS")[0].childNodes)
                                        if len(char_interventions) >=1:
                                            for char_intervention in char_interventions:                                        
                                                if char_intervention.nodeType == minidom.Node.ELEMENT_NODE:
                                                    nodes =  (char_intervention.childNodes)
                                                    for child in nodes:
                                                        if (child.nodeType == minidom.Node.ELEMENT_NODE):
                                                            nodes_level_1 = child.childNodes
                                                            for child_level_1 in nodes_level_1:
                                                                if (child_level_1.nodeType == minidom.Node.TEXT_NODE):
                                                                    text = (child_level_1.data)
                                                                    text  = text.replace("\n","")
                                                                    study_intervention += " " + text
                                                        elif (child.nodeType == minidom.Node.TEXT_NODE):
                                                            text = (child.data)
                                                            text  = text.replace("\n","")
                                                            study_intervention += " " + text
                                        else:
                                            study_intervention = "N/A"
                                    else:
                                        study_intervention = "N/A"

                                    if len(char_included_study[i].getElementsByTagName("CHAR_OUTCOMES")) >=1:
                                        char_outcomes = (char_included_study[i].getElementsByTagName("CHAR_OUTCOMES")[0].childNodes)
                                        if len(char_outcomes) >=1:
                                            for char_outcome in char_outcomes:                                        
                                                if char_outcome.nodeType == minidom.Node.ELEMENT_NODE:
                                                    nodes =  (char_outcome.childNodes)
                                                    for child in nodes:
                                                        if (child.nodeType == minidom.Node.ELEMENT_NODE):
                                                            nodes_level_1 = child.childNodes
                                                            for child_level_1 in nodes_level_1:
                                                                if (child_level_1.nodeType == minidom.Node.TEXT_NODE):
                                                                    text = (child_level_1.data)
                                                                    text  = text.replace("\n","")
                                                                    study_outcome += " " + text
                                                        elif (child.nodeType == minidom.Node.TEXT_NODE):
                                                            text = (child.data)
                                                            text  = text.replace("\n","")
                                                            study_outcome += " " + text
                                        else:
                                            study_outcome = "N/A"
                                    else:
                                        study_outcome = "N/A"

                                    if len(char_included_study[i].getElementsByTagName("CHAR_NOTES")) >=1:
                                        char_notes = (char_included_study[i].getElementsByTagName("CHAR_NOTES")[0].childNodes)
                                        if len(char_notes) >=1:
                                            for char_note in char_notes:                                        
                                                if char_note.nodeType == minidom.Node.ELEMENT_NODE:
                                                    nodes =  (char_note.childNodes)
                                                    for child in nodes:
                                                        if (child.nodeType == minidom.Node.ELEMENT_NODE):
                                                            nodes_level_1 = child.childNodes
                                                            for child_level_1 in nodes_level_1:
                                                                if (child_level_1.nodeType == minidom.Node.TEXT_NODE):
                                                                    text = (child_level_1.data)
                                                                    text  = text.replace("\n","")
                                                                    study_characteristics_notes += " " + text
                                                        elif (child.nodeType == minidom.Node.TEXT_NODE):
                                                            text = (child.data)
                                                            text  = text.replace("\n","")
                                                            study_characteristics_notes += " " + text
                                        else:
                                            study_characteristics_notes = "N/A"    
                                    else:
                                        study_characteristics_notes = "N/A"    

    #                             print (study_method)
    #                             print (study_participants)
    #                             print (study_intervention)
    #                             print (study_outcome)
    #                             print (study_characteristics_notes)

                                review_names_list.append(cochrane_name)
                                study_id_list.append(study_id)
                                study_method_list.append(study_method)
                                study_participants_list.append(study_participants)
                                study_intervention_list.append(study_intervention)
                                study_outcomes_list.append(study_outcome)
                                study_notes_list.append(study_characteristics_notes)
                            
                    
study_with_chars_df = DataFrame({'Review_Name': review_names_list , 
                      'Study_ID': study_id_list,
                      'Study_Method':study_method_list,
                      'Study_Participants':study_participants_list,
                     'Study_Intervention': study_intervention_list,
                     'Study_Outcomes':study_outcomes_list,
                     'Study_Notes':study_notes_list})   

print (len(study_with_chars_df))
display(study_with_chars_df.head(5))

In [ ]:
# study_with_chars_df.to_csv("273_studies_with_characteristics_11182021.csv")
print (len(RCT_filtered_df))
print (len(reference_df))

print ((RCT_filtered_df.columns))
print ((reference_df.columns))

display(RCT_filtered_df.head(2))
display(reference_df.head(2))

In [ ]:
merged_df_2 = pd.merge(
    RCT_filtered_df,
    reference_df,
    how="left",
    left_on=["Study ID","Title"],
    right_on=["Study_ID", "Reference_Title"],
    left_index=False,
    right_index=False)

print (len(merged_df_2))
display(merged_df_2.head(2))

In [ ]:
merged_df_2.to_csv("7226_articles_FULLINFO.csv")

In [ ]:
full_info_df = pd.read_csv("273_studies_FULLINFO.csv")
print (len(full_info_df))
print (full_info_df.columns)
print (len(reference_df))
print (reference_df.columns)

In [ ]:
merged_df = pd.merge(
    reference_df,
    full_info_df,
    how="right",
    right_on=["Study_ID","Title"],
    left_on=["Study_ID","Reference_Title"],
    left_index=False,
    right_index=False)

print (len(merged_df))

In [ ]:
merged_df.to_csv("273_studies_FULLINFO_11_20_2021.csv")